In [87]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/10/08 13:58
# @Author  : Wang Yujia
# @File    : PT_demo.ipynb

# @Description : 1. 用SA试做一下inference of PT model's 3 params 2. 复现table 1的code见PT_demo_table1.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Preparations
## 1.1 Import

In [9]:
import numpy as np
import pandas as pd
import sympy
from sko.SA import SA
from tqdm.notebook import tqdm
import seaborn as sns

## 1.2 Global Settings

In [12]:
# auction settings from Swoopo
#b = 0.75
#v = 169.99
#d = 0.15

# auction settings from paper's Fig.1 left
b= 0.75
v = 27.99
d = 0.01

# cnt_row = data_i.shape[0]
cnt_row = 5
# cnt_n_2 = data_i['cnt_n_2']
cnt_n_2 = [1,1,2,3,1]
#N_i = data_i['N']
N_i = [500,50,100,200,300]
# max duration from Swoopo
max_T = 831

# from paper's Table 5 'All auction'
# alpha = 0.025
# delta = 0.85
# labda = 3.72

# initial params
table_5_M = [0.025,0.85,3.72]
# lower/ upper bound
lb = [-0.3,0.01,0.01]
ub = [0.3, 2, 16]

# paper setting
#T = int((v-b)/d)

# 2. Infer with SA
1. 用SA 求解参数。
2. 定义函数之后，求一个uniq auction下的u和p
3. 然后用p求NLL作为最终的loss

## 2.1 function definition
1. 根据Eq(7)-(9)

In [3]:
def C(t,b):
    return 0.2*t*b

def OMEGA(p,delta):
    return p**delta * ((p**delta + (1-p)**delta)**(-1/delta))

# valuation function
def f(x, alpha):
    return (1-sympy.E**(-alpha*x))/alpha
    # when x < 0, in fact, it shoule be : (-labda)*(1-sympy.E**(alpha*x))/alpha

def f_Equi(t,v,d,b,alpha,labda,delta):
    u = sympy.Symbol('u')

    tmp = v-d*t-C(t-1,b) - b

    func_1 = (labda * f(x=C(t-1, b), alpha=alpha) - labda * OMEGA(u, delta) * f(x=(C(t-1, b) + b), alpha=alpha) + OMEGA(1-u, delta) * f(tmp, alpha))
    func_2 = (-f(x=C(t-1, b), alpha=alpha) + OMEGA(u, delta) * f(x=(C(t-1, b) + b), alpha=alpha) + (1 - OMEGA(u, delta)) * f(-tmp, alpha))

    if(tmp >= 0):
        return sympy.nsolve(func_1,(0,1),solver='bisect', verify=False)
    else:
        return sympy.nsolve(func_2,(0,1),solver='bisect', verify=False)

## 2.2 Loss func
1. 这个函数比较长 没办法，求u需要用SA给的params的值。`loss_func`的返回值是NLL
2. 求解`U_i[t]` which is a array with shape of (max(N)),`U_i[t]`就是paper里的`p_t`
3. for different auction, the duration is N_i[idx] and the value of duration it is various
4. nll计算时乘的`cnt_n_2[idx]`是为了减少计算次数，表示duration相同的auction的数量。

In [10]:
def loss_func(params):

    alpha = params[0]
    delta = params[1]
    labda = params[2]

    # solve for U from Equi. condt.
    U_i = [0] * (max_T + 1)
    U_i[0] = 1
    #P_tmp = [0]*(max_T+1)   # P is what we want but NOT what NLL needs
    #P_tmp[0] = 1

    # tmp = 1
    for t in tqdm(range(1,max_T+1),desc="solve for U"):

        # if((t>1) & (U_i[t-1] > 1-1e-5)):
        #     print("> Break when t == {}, cause U_i[t-1] > 1-1e-6".format(t))
        #     break
        U_i[t] = f_Equi(t, v, d, b, alpha, labda, delta)
        #P_tmp[t] = (1- U_i[t])*tmp
        #tmp = tmp*U_i[t]

    # calculate NLL under this auction setting & PT params
    nll = 0
    if(U_i[0]==1):
        U_i.pop(0)            # because P_tmp[0]=1
    # P_tmp_df = pd.DataFrame(P_tmp,index=np.arange(0,P_tmp.__len__()),columns=['P'],dtype=float)
    U_tmp_df = pd.DataFrame(U_i, index=np.arange(0, U_i.__len__()), columns=['U'], dtype=float)
    # cnt_row = data_i.shape[0]
    for idx in range(0,cnt_row):
        # sum up the log prob among all durations of this auction
        nll += ( np.sum(U_tmp_df[0:(N_i[idx]-1)][:].apply(np.log,axis=1)) + np.log(1-U_tmp_df.iat[(N_i[idx]-1),0]) )* cnt_n_2[idx]

    return float(-nll)

In [132]:
# for SA testing
def demo_func(x):
    alpha = x[0]
    delta = x[1]
    labda = x[2]
    return alpha ** 2 + (labda - 0.05) ** 2 + delta ** 2

-16.679542725154533

In [15]:
print("> Initilizing SA....... \n")
from sko.SA import SABoltzmann
from sko.SA import SACauchy

#sa_cauchy = SACauchy(func=loss_func, x0=table_5_M, T_max=1000, T_min=1e-5, learn_rate=0.01, L=50, max_stay_counter=50,
#                     lb=[-0.3,0.01,0.01], ub=[0.3, 2, 16])

sa_boltzmann = SABoltzmann(func=loss_func, x0=table_5_M, T_max=1000, T_min=1e-5, learn_rate=0.01, L=50, max_stay_counter=50,
                            lb=lb, ub=ub)

#sa = SA(func=loss_func, x0=table_5_M, T_max=1000, T_min=1e-5, L=50, max_stay_counter=50,
#        lb=[-0.3,0.01,0.01], ub=[0.3, 2, 16])

# for SA_Fast:
# [-1.  3. 10.] ok

> Initilizing SA....... 



solve for U:   0%|          | 0/831 [00:00<?, ?it/s]

In [ ]:
print("> Now do SA....... \n")
# best_x, best_y = sa.run()
sa_boltzmann.run()
#sa_cauchy.run()

# print('best_x:', best_x, 'best_y', best_y)
print("SA ends \n")

# sns.lineplot(x = np.arange(0,sa.iter_cycle+1),y=np.array(sa.generation_best_Y))

> Now do SA....... 

-------------- 0_th iteration --------------

---------- 0/L ----------

x_new before clipping:  [0.0045621  0.5558748  3.84665087]
x_new after clipping:  [0.0045621  0.5558748  3.84665087]


solve for U:   0%|          | 0/831 [00:00<?, ?it/s]

y_new - y_current is -13.405098352927581: 
---------- 0/L ----------

---------- 1/L ----------

x_new before clipping:  [0.16324084 0.51542019 3.84990631]
x_new after clipping:  [0.16324084 0.51542019 3.84990631]


solve for U:   0%|          | 0/831 [00:00<?, ?it/s]

y_new - y_current is 40.49307970345589: 
---------- 1/L ----------

---------- 2/L ----------

x_new before clipping:  [0.26678227 0.32447155 3.75189781]
x_new after clipping:  [0.26678227 0.32447155 3.75189781]


...

solve for U:   0%|          | 0/831 [00:00<?, ?it/s]

y_new - y_current is -1.5267993463398142: 
---------- 7/L ----------

---------- 8/L ----------

x_new before clipping:  [0.32191141 1.10854657 3.36177253]
x_new after clipping:  [0.3        1.10854657 3.36177253]


solve for U:   0%|          | 0/831 [00:00<?, ?it/s]

y_new - y_current is 3.0782872483551387: 
---------- 8/L ----------

---------- 9/L ----------

x_new before clipping:  [-0.00375933  1.22798997  3.41353145]
x_new after clipping:  [-0.00375933  1.22798997  3.41353145]


solve for U:   0%|          | 0/831 [00:00<?, ?it/s]

In [ ]:
# alpha_g = -0.1
# def g(x):
#     return (1-np.exp(-alpha_g*x))/alpha_g
#
# x_g = pd.DataFrame(np.arange(0,100),index=(np.arange(0,100)),columns=['x'])
# x_g['y'] = x_g.apply(g,axis=1)
# sns.scatterplot(x = x_g['x'],y=x_g['y'])